# Metadata for reviews

In [27]:
import pandas as pd
import ast, os, csv

In [4]:
j = ast.literal_eval("['this', 'is', 'a', 'list']")
j[2]

'a'

In [13]:
titles = pd.read_csv('../fileindex.csv')
titles.head()

,title,filename,description,estimateddaterange
0,The Athenaeum,athenaeum,"UK magazine, lots of short book reviews",1830-1921
1,Booklist and Subscription Books Bulletin,booklistandsubscription,"US mag, published by American Library Association",1905-1933
2,Book Review Digest,brd,"our sweetest source of book review text, trans...",1906-2002
3,Publishers Weekly,publishersweekly,"has book prices and paper/cloth, among other t...",1873-2007
4,The Atlantic,atlantic,US mag,1857-1963


In [33]:
titledict = dict()

allvols = set()
with open('tedsnewvolids.txt', encoding = 'utf-8') as f:
    for line in f:
        allvols.add(line.strip())

def mine_ris(filename, thelist):
    rispath = '../citations/' + filename + '.ris'
    missing = 0
    found = 0
    voldict = dict()
    
    with open(rispath, encoding = 'utf-8') as f:
        for line in f:
            if not line.startswith('UR'):
                continue
            else:
                parts = line.strip().split('/')
                useful = parts[-1].split(' ', 1)
                volid = useful[0]
                if len(useful) > 1:
                    enumcron = useful[1]
                else:
                    enumcron = ''
                    
                if volid not in thelist:
                    missing += 1
                else:
                    found += 1
            
                voldict[volid] = enumcron
    
    return voldict, missing, found

outlines = []

for filename in titles.filename:
    filepath = '../' + filename + '.txt'
    
    if not os.path.isfile(filepath):
        print(filepath, 'missing summary file')
        thelist = []
    else:
        with open(filepath, encoding = 'utf-8') as f:
            stringlist = f.read()
            thelist = ast.literal_eval(stringlist)
            titledict[filename] = thelist
        
    rispath = '../citations/' + filename + '.ris'
    missing = 0
    found = 0
    
    rispath = '../citations/' + filename + '.ris'
    if os.path.isfile(rispath):
        voldict, missing, found = mine_ris(filename, thelist)
    elif os.path.isfile('../citations/' + filename + '1.ris'):
        voldict = dict()
        for partnum in ['1', '2']:
            missing = 0
            found = 0
            v, m, f = mine_ris(filename + partnum, thelist)
            missing += m
            found += f
            for key, value in v.items():
                voldict[key] = value
    else:
        voldict = dict()
        missing = 0
        found = 0
    
    print(filename, len(thelist), missing, found)
    
    for volid in set(thelist):
        if volid in voldict:
            enumcron = voldict[volid]
        else:
            enumcron = ''
            
        o = [volid, filename, titles.loc[titles.filename == filename, 'title'].values[0], enumcron]
        outlines.append(o)

with open('volmeta.csv', mode = 'w', encoding = 'utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['volid', 'filename', 'title', 'enumcron'])
    for o in outlines:
        writer.writerow(o)
    
print(len(outlines))   
print(len(allvols))

athenaeum 361 77 14
booklistandsubscription 69 2 61
brd 186 1 72
publishersweekly 554 2 130
atlantic 306 3 78
thenewyorker 381 419 271
americanlittle 23 6 7
commonwealthlittle 9 1 9
spectator 408 30 381
nytbr 268 4 140
rgpl 110 1 71
thenewrepublic 52 27 47
blackwoods 1601 46 512
../bookmanlo.txt missing summary file
bookmanlo 0 102 0
bookmanny 435 53 216
comprehensivelittle 8 1 8
subjectindex 46 38 42
britishhumanities 34 5 31
19creadersguide 298 11 0
fortnightly 298 68 221
scrutiny 26 17 10
horizon 31 1 18
newstatesman 68 4 15
edinburgh 1398 112 0
tls 25 1 14
../pooles.txt missing summary file
pooles 0 0 0
6995
6775
